# Import and Load Data

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
from astropy.table import Table
from astropy.io import fits
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm

## modules I am still trying to learn how to use
import numexpr
import bottleneck

## Functions

In [2]:
## function to extract fits and hdf data
def load_true():
    """This is a function that takes fits and hdf data and return them to pandas dataframe"""

    # detect = fits.open('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits')[-1]
    phot   = fits.open('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits')[1]
    id_balrog = fits.open('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits')

    #load true catalogs in pandas
    # detectp_table=Table.read('/lsst/troxel/balrog/balrog_detection_catalog-v1.2.fits').to_pandas()
    photdf=Table.read('/lsst/troxel/balrog/balrog_matched_catalog_flatten.fits').to_pandas()
    id_balrogdf = Table.read('/lsst/troxel/balrog/balrog_mcal_bal_ids_v1.2.fits').to_pandas()
    
    photdf.drop_duplicates(['bal_id'],inplace=True)
    
    photdf.set_index('bal_id',inplace=True,verify_integrity=True)

    return(photdf)


def load_mcal():
    #all objects riz
#     mcal0= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-v1.2.h5','r')['catalog']['unsheared'] #with neighbor
#     mcal1= h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-v1.2.h5','r')['catalog']['unsheared'] #no neighbor
    ##only matched balrog object riz
    mcal2=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal3=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-riz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    #only matched balrog object griz
    mcal4=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-mcal-1.3.h5','r')['catalog']['unsheared'] #with neighbor
    mcal5=h5py.File('/lsst/troxel/balrog/balrog_mcal_stack-y3v02-0-griz-noNB-mcal-1.3.h5','r')['catalog']['unsheared'] #no neighbor
    
    mcal_list=[mcal2,mcal3,mcal4,mcal5]
    return (mcal_list)

In [3]:
## function to convert 
def get_df_list(mcal_list):
    mcal_df_list =[]
    for catalog in mcal_list:
        local_df=pd.DataFrame()
        for key in catalog.keys():
            local_df[key]=np.array(catalog[key][:]).byteswap().newbyteorder()
        if 'bal_id' in local_df.keys():
            local_df.set_index('bal_id',inplace=True,verify_integrity=True)
        mcal_df_list.append(local_df)
    return(mcal_df_list)

In [4]:
photdf=load_true()

In [5]:
mcal_df_list=get_df_list(load_mcal())

## List of dtypes in mcal

In [6]:
mcal_df_list[2].dtypes

R11                    >f8
R12                    >f8
R21                    >f8
R22                    >f8
T                  float64
T_err              float64
coadd_object_id      int64
covmat_0_1         float64
covmat_1_1         float64
covmat_2_2         float64
dec                float64
e_1                float64
e_2                float64
flags                int32
flux_err_g         float64
flux_err_i         float64
flux_err_r         float64
flux_err_z         float64
flux_g             float64
flux_i             float64
flux_r             float64
flux_z             float64
mask_frac          float64
mcal_psf_T         float64
mcal_psf_e1        float64
mcal_psf_e2        float64
nimage_tot_g         int32
nimage_tot_i         int32
nimage_tot_r         int32
nimage_tot_z         int32
nimage_use_g         int32
nimage_use_i         int32
nimage_use_r         int32
nimage_use_z         int32
psf_T              float64
psf_e1             float64
psf_e2             float64
r

## List of dtypes in phot

In [7]:
photdf.dtypes

true_id                    int32
true_number                int32
true_flags                 int32
true_obj_flags             int32
true_cm_mof_flags          int32
true_ra                  float64
true_dec                 float64
true_cm_T                float64
true_cm_T_err            float64
true_cm_s2n_r            float64
true_cm_flags              int32
true_cm_fracdev          float64
true_cm_TdByTe           float32
true_in_VHS_footprint      int32
true_mag_auto_det        float32
true_det_number            int32
true_tilename             object
meas_id                    int64
meas_number                int32
meas_ra                  float64
meas_dec                 float64
meas_fofid                 int64
meas_flags                 int32
meas_time_last_fit       float64
meas_box_size              int16
meas_obj_flags             int32
meas_mask_frac           float64
meas_psfrec_T            float64
meas_cm_flags              int32
meas_cm_T                float64
          

## sample of mcal data

In [8]:
mcal_df_list[3].head()

,R11,R12,R21,R22,T,T_err,coadd_object_id,covmat_0_1,covmat_1_1,covmat_2_2,...,nimage_use_g,nimage_use_i,nimage_use_r,nimage_use_z,psf_T,psf_e1,psf_e2,ra,size_ratio,snr
bal_id,,,,,,,,,,,,,,,,,,,,,
100304018326232,1.416846,0.872972,0.426857,0.995899,0.554205,0.051718,337,-0.000059,0.000793,0.001003,...,3,4,4,4,0.459272,0.005778,0.008893,45.745366,1.206704,29.133429
100304018321548,0.823930,-1.786024,0.167303,0.362565,0.059780,0.018368,345,0.001073,0.011678,0.012190,...,2,3,2,4,0.522193,0.002164,0.010059,46.178671,0.114479,27.278783
10030401832834,0.387232,-0.162681,-0.212697,0.681578,0.180421,0.019096,355,0.000255,0.002476,0.002242,...,2,4,3,4,0.518533,-0.004169,0.023148,46.317107,0.347944,41.197880
100304018328103,1.126121,-0.155307,0.074416,0.661508,1.599951,0.065488,393,0.000001,0.000288,0.000287,...,2,4,3,4,0.520347,-0.004125,0.024472,46.299203,3.074778,34.434555
100304018322349,1.469382,-2.897695,-1.852615,-4.158914,-0.004546,0.001035,399,0.003697,0.015358,0.015758,...,4,5,5,5,0.497255,0.017866,0.009877,45.865877,-0.009142,376.417275


# Data manipulation

## Constants

In [9]:
SNR_CUT = 10
SIZE_RATIO_CUT = 0.5
T_CUT = 1000

## Filter by SNR T and size_ratio

In [10]:
def filter_by_snr():
    for i,catalog in enumerate(mcal_df_list):
        mcal_df_list[i]=catalog[(catalog['snr']>SNR_CUT) & (catalog['T']<T_CUT) & (catalog['size_ratio']>SIZE_RATIO_CUT)]

In [11]:
filter_by_snr()

## Intersects paris of catalogs by bal_id. We only want objects existing in true catalog, MOF and woMOF  

In [12]:
index_2_3=mcal_df_list[0].index.intersection(mcal_df_list[1].index)
index_4_5=mcal_df_list[2].index.intersection(mcal_df_list[3].index)

index_2_3_true=index_2_3.intersection(photdf.index).sort_values()
index_4_5_true=index_4_5.intersection(photdf.index).sort_values()

In [13]:
mcal_df_list[0]=mcal_df_list[0].reindex(index_2_3_true);
mcal_df_list[1]=mcal_df_list[1].reindex(index_2_3_true);
mcal_df_list[2]=mcal_df_list[2].reindex(index_4_5_true);
mcal_df_list[3]=mcal_df_list[3].reindex(index_4_5_true);

In [14]:
for catalog in mcal_df_list[0:4]:
    print(len(catalog.index))
#     print(catalog.T.isnull().any())

1098637
1098637
867808
867808


# Getting dictionary arrays

In [20]:
cols=phot[1].columns
true_data = phot[1].data

true_dict = get_true_dict(true_data)

NameError: name 'phot' is not defined

In [ ]:
true_dict_unique=get_unique_dict(true_dict);

In [ ]:
true_dict_unique_sorted=sort_dict_by_bal(true_dict_unique)

In [ ]:
def get_positive_g_mask(true_dict):
    g1_mask = np.ma.masked_greater(true_dict['true_cm_g_1'],0)
    g2_mask = np.ma.masked_greater(true_dict['true_cm_g_2'],0)
    g_mask = np.logical_and(g1_mask.mask, g2_mask.mask)
    return(g_mask)

In [ ]:
def get_negative_g_mask(true_dict):
    g1_mask = np.ma.masked_less(true_dict['true_cm_g_1'],0)
    g2_mask = np.ma.masked_less(true_dict['true_cm_g_2'],0)
    g_mask = np.logical_and(g1_mask.mask, g2_mask.mask)
    return(g_mask)

In [ ]:
positive_g_mask=get_positive_g_mask(true_dict_unique_sorted)
negative_g_mask=get_negative_g_mask(true_dict_unique_sorted)

In [ ]:
true_positive_g=mask_catalog_arrays([true_dict_unique_sorted],[positive_g_mask])[0]
true_negative_g=mask_catalog_arrays([true_dict_unique_sorted],[negative_g_mask])[0]

In [ ]:
positive_filtered=[]
positive_true=[]
for i,catalog in enumerate(relavant_catalogs):
    positive_mask_true,positive_mask_mcal = get_intersected_mask(true_positive_g,catalog)
    positive_filtered.append(mask_catalog_arrays([catalog],[positive_mask_mcal])[0])
    positive_true.append(mask_catalog_arrays([true_positive_g],[positive_mask_true])[0])

In [ ]:
negative_filtered=[]
negative_true=[]
for i,catalog in enumerate(relavant_catalogs):
    negative_mask_true,negative_mask_mcal = get_intersected_mask(true_negative_g,catalog)
    negative_filtered.append(mask_catalog_arrays([catalog],[negative_mask_mcal])[0])
    negative_true.append(mask_catalog_arrays([true_negative_g],[negative_mask_true])[0])

In [ ]:
print (len(negative_filtered))

# Intersect mcal2/3 mcal4/5

In [ ]:
def intersect_the_same_catalog():
    duo_intersected_array_positive=[]
    duo_intersected_array_negative=[]
    for i in [0,2]:
        (mask1,mask2)=get_intersected_mask(positive_filtered[i],positive_filtered[i+1])
        (mask3,mask4)=get_intersected_mask(negative_filtered[i],negative_filtered[i+1])
        mask_array=[mask1,mask2,mask3,mask4]
        duo_intersected_array_positive.append(mask_catalog_arrays([positive_filtered[i]],[mask_array[0]])[0])
        duo_intersected_array_positive.append(mask_catalog_arrays([positive_filtered[i+1]],[mask_array[1]])[0])
        duo_intersected_array_negative.append(mask_catalog_arrays([negative_filtered[i]],[mask_array[2]])[0])
        duo_intersected_array_negative.append(mask_catalog_arrays([negative_filtered[i+1]],[mask_array[3]])[0])
    return(duo_intersected_array_positive,duo_intersected_array_negative)

In [ ]:
(nor_positive,nor_negative) = intersect_the_same_catalog()

In [ ]:
positive_filtered=[]
positive_true=[]
for i,catalog in enumerate(nor_positive):
    positive_mask_true,positive_mask_mcal = get_intersected_mask(true_positive_g,catalog)
    positive_filtered.append(mask_catalog_arrays([catalog],[positive_mask_mcal])[0])
    positive_true.append(mask_catalog_arrays([true_positive_g],[positive_mask_true])[0])

In [ ]:
negative_filtered=[]
negative_true=[]
for i,catalog in enumerate(nor_negative):
    negative_mask_true,negative_mask_mcal = get_intersected_mask(true_negative_g,catalog)
    negative_filtered.append(mask_catalog_arrays([catalog],[negative_mask_mcal])[0])
    negative_true.append(mask_catalog_arrays([true_negative_g],[negative_mask_true])[0])

In [ ]:
for catalog in nor_negative:
    print (len(catalog['bal_id']))
print (len(negative_true[0]['bal_id']))

In [ ]:
print (np.equal(nor_positive[2]['e_1'],nor_positive[3]['e_1']))

In [ ]:
print (np.equal(negative_true[0]['true_cm_g_1'],negative_true[1]['true_cm_g_1']).all())

In [ ]:
positive_filtered=nor_positive
negative_filtered=nor_negative

In [ ]:
def get_R(mcal,true):
    e1=np.mean(mcal['e_1'])
    e2=np.mean(mcal['e_2'])
    g1=np.mean(true['true_cm_g_1'])
    g2=np.mean(true['true_cm_g_2'])
    r11=e1/g1
    r12=e1/g2
    r21=e2/g1
    r22=e2/g2
    return([[r11,r12],[r21,r22]],[[e1,e2],[g1,g2]])

In [ ]:
R_array_positive=[]
eg_array_positive=[]
for i in range(4):
    R_array_positive.append(get_R(positive_filtered[i],positive_true[i])[0])
    eg_array_positive.append(get_R(positive_filtered[i],positive_true[i])[1])

In [ ]:
R_array_negative=[]
eg_array_negative=[]
for i in range(4):
    R_array_negative.append(get_R(negative_filtered[i],negative_true[i])[0])
    eg_array_negative.append(get_R(negative_filtered[i],negative_true[i])[1])

In [ ]:
real_R_array_positive=[]
for i in range(4):
    r11=np.mean(positive_filtered[i]['R11'])
    r12=np.mean(positive_filtered[i]['R12'])
    r21=np.mean(positive_filtered[i]['R21'])
    r22=np.mean(positive_filtered[i]['R22'])
    real_R_array_positive.append([[r11,r12],[r21,r22]])

In [ ]:
real_R_array_negative=[]
for i in range(4):
    r11=np.mean(negative_filtered[i]['R11'])
    r12=np.mean(negative_filtered[i]['R12'])
    r21=np.mean(negative_filtered[i]['R21'])
    r22=np.mean(negative_filtered[i]['R22'])
    real_R_array_negative.append([[r11,r12],[r21,r22]])

In [ ]:
for i,R_array in enumerate(R_array_negative):
    e1_error=np.std(negative_filtered[i]['e_1'])/np.sqrt(float(len(negative_filtered[i]['e_1'])))
    e2_error=np.std(negative_filtered[i]['e_2'])/np.sqrt(float(len(negative_filtered[i]['e_2'])))
    
    g1_error=np.std(negative_true[i]['true_cm_g_1'])/np.sqrt(float(len(negative_true[i]['true_cm_g_1'])))
    g2_error=np.std(negative_true[i]['true_cm_g_2'])/np.sqrt(float(len(negative_true[i]['true_cm_g_2'])))
    
    e1_mean=np.mean(negative_filtered[i]['e_1'])
    e2_mean=np.mean(negative_filtered[i]['e_2'])
    
    g1_mean=np.mean(negative_true[i]['true_cm_g_1'])
    g2_mean=np.mean(negative_true[i]['true_cm_g_2'])
    
    R11_error=np.sqrt((1/g1_mean)**2 * e1_error**2 + (e1_mean/(g1_mean**2))**2*g1_error**2)
    R22_error=np.sqrt((1/g2_mean)**2 * e2_error**2 + (e2_mean/(g2_mean**2))**2*g2_error**2)
    
    R_11_mean=np.mean(negative_filtered[i]['R11'])
    R_22_mean=np.mean(negative_filtered[i]['R22'])
    R_11_std=np.std(negative_filtered[i]['R11'])
    R_22_std=np.std(negative_filtered[i]['R22'])

    
    R_11_read_error=R_11_std/np.sqrt(float(len(negative_filtered[i]['R11'])))
    R_22_read_error=R_11_std/np.sqrt(float(len(negative_filtered[i]['R22'])))

    
    
    g_prime_1=np.mean(negative_filtered[i]['e_1'])/R_11_mean
    g_prime_2=np.mean(negative_filtered[i]['e_2'])/R_22_mean
    
    g_prime_1_error=np.sqrt((1/R_11_mean)**2 * e1_error**2 + (e1_mean/R_11_mean**2)**2*R_11_read_error**2)
    g_prime_2_error=np.sqrt((1/R_22_mean)**2 * e2_error**2 + (e2_mean/R_22_mean**2)**2*R_22_read_error**2)
    print("For mcal{} with negative g, measured R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*row1))
    print("<e1>={:5.3e}, <e2>={:5.3e}, <g1>={:5.3e}, <g2>={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(*eg_array_negative[i][0],*eg_array_negative[i][1],real_R_array_negative[i][0][0],real_R_array_negative[i][1][1]))
    print("d_e1={:5.3e}, d_e2={:5.3e}".format(e1_error,e2_error))
    print("R11'_error={:5.3e} R22'_error={:5.3e}".format(R11_error,R22_error))
    print("\n")
    print ("g1'={:5.3e}, g2'={:5.3e},".format(g_prime_1,g_prime_2)+" g1'_error={:5.3e}, g2'_error={:5.3e}".format(g_prime_1_error,g_prime_2_error))
    print("\n")
    print("Theoretical R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*real_R_array_negative[i][j]))
    print ("R11'={:5.3e}, R22'={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(R_array_negative[i][0][0],R_array_negative[i][1][1],real_R_array_negative[i][0][0],real_R_array_negative[i][1][1]))
    print("-----------------------------------------------------------------")
    print("\n")


In [ ]:
for i,R_array in enumerate(R_array_positive):
    e1_error=np.std(positive_filtered[i]['e_1'])/np.sqrt(float(len(positive_filtered[i]['e_1'])))
    e2_error=np.std(positive_filtered[i]['e_2'])/np.sqrt(float(len(positive_filtered[i]['e_2'])))
    g1_error=np.std(positive_true[i]['true_cm_g_1'])/np.sqrt(float(len(positive_true[i]['true_cm_g_1'])))
    g2_error=np.std(positive_true[i]['true_cm_g_2'])/np.sqrt(float(len(positive_true[i]['true_cm_g_2'])))
    e1_mean=np.mean(positive_filtered[i]['e_1'])
    e2_mean=np.mean(positive_filtered[i]['e_2'])
    g1_mean=np.mean(positive_true[i]['true_cm_g_1'])
    g2_mean=np.mean(positive_true[i]['true_cm_g_2'])
    R11_error=np.sqrt((1/g1_mean)**2 * e1_error**2 + (e1_mean/(g1_mean**2))**2*g1_error**2)
    R22_error=np.sqrt((1/g2_mean)**2 * e2_error**2 + (e2_mean/(g2_mean**2))**2*g2_error**2)
    
    R_11_mean=np.mean(positive_filtered[i]['R11'])
    R_22_mean=np.mean(positive_filtered[i]['R22'])
    R_11_std=np.std(negative_filtered[i]['R11'])
    R_22_std=np.std(negative_filtered[i]['R22'])
    
    R_11_read_error=R_11_std/np.sqrt(float(len(positive_filtered[i]['R11'])))
    R_22_read_error=R_22_std/np.sqrt(float(len(positive_filtered[i]['R22'])))
    
    g_prime_1=np.mean(positive_filtered[i]['e_1'])/R_11_mean
    g_prime_2=np.mean(positive_filtered[i]['e_2'])/R_22_mean
    
    g_prime_1_error=np.sqrt((1/R_11_mean)**2 * e1_error**2 + (e1_mean/(R_11_mean**2))**2*R_11_read_error**2)
    g_prime_2_error=np.sqrt((1/R_22_mean)**2 * e2_error**2 + (e2_mean/(R_22_mean**2))**2*R_22_read_error**2)

    

    print("For mcal{} with positive g, measured R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*row1))
    print("<e1>={:5.3e}, <e2>={:5.3e}, <g1>={:5.3e}, <g2>={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(*eg_array_positive[i][0],*eg_array_positive[i][1],real_R_array_positive[i][0][0],real_R_array_positive[i][1][1]))
    print("d_e1={:5.3e}, d_e2={:5.3e}".format(e1_error,e2_error))
    print("R11'_error={:5.3e} R22'_error={:5.3e}".format(R11_error,R22_error))
    print("\n")
    print ("g1'={:5.3e}, g2'={:5.3e},".format(g_prime_1,g_prime_2)+" g1'_error={:5.3e}, g2'_error={:5.3e}".format(g_prime_1_error,g_prime_2_error))
    
    print("Theoretical R is".format(i+2))
    for j,row1 in enumerate(R_array):
        print("|{:5.3e} {:5.3e}|".format(*real_R_array_positive[i][j]))
    print ("R11'={:5.3e}, R22'={:5.3e}, R11={:5.3e}, R22={:5.3e}".format(R_array_positive[i][0][0],R_array_positive[i][1][1],real_R_array_positive[i][0][0],real_R_array_positive[i][1][1]))
    print("-----------------------------------------------------------------")

In [ ]:
print ("For positive_g")
for i in [0,2]:
    print ("The difference beween R matrice in mcal{:5.3e} and mcal{:5.3e} is:".format(i+2,i+3))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_positive[i][j])-np.array(R_array_positive[i+1][j]))))

In [ ]:
print ("For negative g")
for i in [0,2]:
    print ("The difference beween R matrice in mcal{} and mcal{} is:".format(i+2,i+3))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_negative[i][j])-np.array(R_array_negative[i+1][j]))))

In [ ]:
print ("For positive g")
for i in range(4):
    print("For mcal{}, the diffrence between measured and theoretical R is".format(i+2))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_positive[i][j])-np.array(real_R_array_positive[i][j])))) 

In [ ]:
print ("For negative g")
for i in range(4):
    print("For mcal{}, the diffrence between measured and theoretical R is".format(i+2))
    for j in range(2):
        print ("|{:5.3e} {:5.3e}|".format(*(np.array(R_array_negative[i][j])-np.array(real_R_array_negative[i][j])))) 

# Fitting e_i = m_i * g + c

In [ ]:
intersected_masks=[get_intersected_mask(true_dict_unique_sorted,catalog) for catalog in relavant_catalogs]

In [ ]:
intersected_masks_for_mcal=[mask[1] for mask in intersected_masks]
intersected_masks_for_true=[mask[0] for mask in intersected_masks]

In [ ]:
intersected_clipped_array=mask_catalog_arrays(relavant_catalogs,intersected_masks_for_mcal)

In [ ]:
four_true_dicts=[true_dict_unique_sorted for i in range(4)]

In [ ]:
intersected_clipped_true=mask_catalog_arrays(four_true_dicts,intersected_masks_for_true)

In [ ]:
def get_cm(true_dict,mcal_dict):
        y1=mcal_dict['e_1']
        x1=true_dict['true_cm_g_1']
        y2=mcal_dict['e_2']
        x2=true_dict['true_cm_g_2']
        m1,c1=np.polyfit(x1,y1,1)
        m2,c2=np.polyfit(x2,y2,1)
        return([[m1,c1],[m2,c2]])

In [ ]:
cm_array=[get_cm(intersected_clipped_true[i],intersected_clipped_array[i]) for i in range(4)]

In [ ]:
cm_array_positive=[get_cm(positive_true[i],positive_filtered[i]) for i in range(4)]

In [ ]:
cm_array_negative=[get_cm(negative_true[i],negative_filtered[i]) for i in range(4)]

In [ ]:
def print_cm(cm_array):
    for i,matrix in enumerate(cm_array):
        print("For mcal{}, m1={:10.3e},c1={:10.3e},m2={:10.3e},c2={:10.3e}".format((i+2),*matrix[0],*matrix[1]))

In [ ]:
print_cm(cm_array)

In [ ]:
print_cm(cm_array_positive)

In [ ]:
print_cm(cm_array_negative)

In [ ]:
x1_array=[true_dict['true_cm_g_1'] for true_dict in intersected_clipped_true]
x2_array=[true_dict['true_cm_g_2'] for true_dict in intersected_clipped_true]
y1_array=[mcal_dict['e_1'] for mcal_dict in intersected_clipped_array]
y2_array=[mcal_dict['e_2'] for mcal_dict in intersected_clipped_array]

In [ ]:
x3_array=[true_dict['true_cm_g_1'] for true_dict in positive_true]
x4_array=[true_dict['true_cm_g_2'] for true_dict in positive_true]
y3_array=[mcal_dict['e_1'] for mcal_dict in positive_filtered]
y4_array=[mcal_dict['e_2'] for mcal_dict in positive_filtered]

In [ ]:
x5_array=[true_dict['true_cm_g_1'] for true_dict in negative_true]
x6_array=[true_dict['true_cm_g_2'] for true_dict in negative_true]
y5_array=[mcal_dict['e_1'] for mcal_dict in negative_filtered]
y6_array=[mcal_dict['e_2'] for mcal_dict in negative_filtered]

In [ ]:
colors=(0,0,0)
area=0.01
def plot_scatter(x,y,cm):
    plt.scatter(x,y,s=area,c=colors,alpha=0.5)
    x_range=np.linspace(0,1)
    plt.plot(x_range,cm[0]*x_range+cm[1])

In [ ]:
for i in range(4):
    plt.figure()
    plot_scatter(x1_array[i],y1_array[i],cm_array[i][0])
    plt.title("mcal{} g_1 e_1".format(i+2))
    plt.savefig("scatter/mcal{}g1e1".format(i+2),dpi=300)
    plt.close()
    plt.figure()
    plot_scatter(x2_array[i],y2_array[i],cm_array[i][1])
    plt.title("mcal{} g_2 e_2".format(i+2))
    plt.savefig("scatter/mcal{}g2e2".format(i+2),dpi=300)
    plt.close()

In [ ]:
for i in range(4):
    plt.figure()
    plot_scatter(x3_array[i],y3_array[i],cm_array_positive[i][0])
    plt.title("mcal{} g_1 e_1_positive".format(i+2))
    plt.savefig("scatter/mcal{}g1e1_positive".format(i+2),dpi=300)
    plt.close()
    plt.figure()
    plot_scatter(x4_array[i],y4_array[i],cm_array[i][1])
    plt.title("mcal{} g_2 e_2_positive".format(i+2))
    plt.savefig("scatter/mcal{}g2e2_positive".format(i+2),dpi=300)
    plt.close()

In [ ]:
for i in range(4):
    plt.figure()
    plot_scatter(x3_array[i],y3_array[i],cm_array_positive[i][0])
    plt.title("mcal{} g_1 e_1_negative".format(i+2))
    plt.savefig("scatter/mcal{}g1e1_negative".format(i+2),dpi=300)
    plt.close()
    plt.figure()
    plot_scatter(x4_array[i],y4_array[i],cm_array[i][1])
    plt.title("mcal{} g_2 e_2_negative".format(i+2))
    plt.savefig("scatter/mcal{}g2e2_negative".format(i+2),dpi=300)
    plt.close()

In [ ]:
for i in [0,2]:
    print(i)

# healpix

In [ ]:
from astropy_healpix import HEALpix
from astropy.coordinate import icrs

